In [2]:
import pandas as pd

In [13]:

# Load the raw datasets
raw_data_path = r"C:\Users\brvn\Documents\github\projects\HC\tcc_brvn\data\raw\2020-2023.csv"
raw_df = pd.read_csv(raw_data_path, delimiter=';')

raw_data_2016 = r"C:\Users\brvn\Documents\github\projects\HC\tcc_brvn\data\raw\2016-2019.csv"  # Correct the path to 2016-2019 data
df_2016 = pd.read_csv(raw_data_2016, delimiter=";")



In [10]:
import pandas as pd

def clean_and_convert_to_time_series(raw_data_path):
    # Load the raw dataset
    raw_df = pd.read_csv(raw_data_path, delimiter=';')
    
    # Define a function to clean the raw data
    def clean_raw_data(df):
        # Remove the 'Total' column
        df = df.drop(columns=['Total'])
        # Filter out rows where 'SubGrup proced' is not equal to 'Total'
        df = df[df['SubGrup proced'] != 'Total']
        return df

    # Clean the dataset
    cleaned_df = clean_raw_data(raw_df)
    
    # Define the columns to keep as identifiers (e.g., 'SubGrup proced')
    id_vars = ['SubGrup proced']
    
    # Use pd.melt to reshape the DataFrame into a time series format
    ts_df = pd.melt(cleaned_df, id_vars=id_vars, var_name='Date', value_name='Value')
    
    # Create a custom function to parse the date strings into datetime objects
    def parse_custom_date(date_str):
        # Define a mapping of non-English month abbreviations to English
        month_mapping = {
            'Jan': 'January',
            'Fev': 'February',
            'Mar': 'March',
            'Abr': 'April',
            'Mai': 'May',
            'Jun': 'June',
            'Jul': 'July',
            'Ago': 'August',
            'Set': 'September',
            'Out': 'October',
            'Nov': 'November',
            'Dez': 'December'
        }
        
        # Check if the date_str is not 'Total' before splitting
        if date_str != 'Total':
            # Split the input string into month and year parts
            month_str, year_str = date_str.split('/')
            # Replace the non-English month abbreviation with the English one
            month_str = month_mapping.get(month_str, month_str)
            # Convert the date string into a datetime object
            return pd.to_datetime(f"{month_str}/{year_str}", format='%B/%Y')
        
        return date_str  # Return 'Total' as-is

    # Apply the custom date parsing function to the 'Date' column
    ts_df['Date'] = ts_df['Date'].apply(parse_custom_date)
    
    # Set the 'Date' column as the index
    ts_df.set_index('Date', inplace=True)
    
    return ts_df


,SubGrup proced,Value
Date,,
2020-01-01,0202 Diagnostico em laboratorio clinico,43209
2020-01-01,0301 Consultas / Atendimentos / Acompanhamentos,23591
2020-02-01,0202 Diagnostico em laboratorio clinico,40841
2020-02-01,0301 Consultas / Atendimentos / Acompanhamentos,21853
2020-03-01,0202 Diagnostico em laboratorio clinico,29159
...,...,...
2023-06-01,0301 Consultas / Atendimentos / Acompanhamentos,24559
2023-07-01,0202 Diagnostico em laboratorio clinico,41197
2023-07-01,0301 Consultas / Atendimentos / Acompanhamentos,24287


In [19]:
ts_2020 = clean_and_convert_to_time_series(raw_data_path)
ts_2016 = clean_and_convert_to_time_series(raw_data_2016)


In [20]:
ts_df = pd.concat([ts_2016, ts_2020])
ts_df

,SubGrup proced,Value
Date,,
2017-01-01,0202 Diagnostico em laboratorio clinico,38838
2017-01-01,0301 Consultas / Atendimentos / Acompanhamentos,21215
2017-02-01,0202 Diagnostico em laboratorio clinico,30560
2017-02-01,0301 Consultas / Atendimentos / Acompanhamentos,17021
2017-03-01,0202 Diagnostico em laboratorio clinico,30194
...,...,...
2023-06-01,0301 Consultas / Atendimentos / Acompanhamentos,24559
2023-07-01,0202 Diagnostico em laboratorio clinico,41197
2023-07-01,0301 Consultas / Atendimentos / Acompanhamentos,24287


In [21]:
# Save the combined time series dataset to a CSV file
output_csv_path = r"C:\Users\brvn\Documents\github\projects\HC\tcc_brvn\data\processed\processed-time-series.csv"
ts_df.to_csv(output_csv_path, sep=';', index=True)  # Use the appropriate path and separator

# Print a confirmation message
print(f"The combined time series dataset has been saved to: {output_csv_path}")


The combined time series dataset has been saved to: C:\Users\brvn\Documents\github\projects\HC\tcc_brvn\data\processed\processed-time-series.csv
